In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from difflib import SequenceMatcher

def similarity(a, b):
    if a is None or b is None:
        return 0
    return SequenceMatcher(None, str(a), str(b)).ratio()

## Load Datasets

In [ ]:
import sys
sys.path.append("../src/")

In [ ]:
from database.leads import s3_repository

s3 = s3_repository.s3

In [ ]:
def fetch_object_s3(bucket, obj_key):
    """
    Tries to read an object from S3.
    :return: s3 object
    """
    obj = None
    try:
        remote_dataset = s3.get_object(Bucket=bucket, Key=obj_key)
    except botocore.exceptions.ClientError as e:
        log.warning(
            f"{e.response['Error']['Code']}: {e.response['Error']['Message']}"
            if "Error" in e.response
            else f"Error while getting object s3://{bucket}/{obj_key}"
        )

    if remote_dataset is None or "Body" not in remote_dataset:
        log.error(f"Couldn't find dataset in S3 bucket {bucket} and key {obj_key}")
        return
    else:
        source = remote_dataset["Body"]

    try:
        df = pd.read_csv(source)
    except FileNotFoundError:
        log.error("Error: Could not find input file for Pipeline.")

    return df

In [ ]:
df = fetch_object_s3("amos-training-data", "100k_historic_enriched.csv")

In [ ]:
#df_10k = pd.read_csv("s3://amos-training-data/10k_historic_enriched_with_regionalatlas.csv", index_col="Unnamed: 0")
#df_1k = pd.read_csv("s3://amos-training-data/historic_enriched.csv")
#df_1k.info()

In [ ]:
df.head()

In [ ]:
historic_df = fetch_object_s3("amos-training-data", "AMOS export historic data Dec 6, 23.csv")

In [ ]:
#df_90k = pd.read_csv("s3://amos-training-data/90k_historic_enriched.csv", index_col="Unnamed: 0")

### Merge 90k and 10k enriched samples

In [ ]:
# df = pd.concat([df_10k, df_90k])

In [ ]:
# df.info()

In [ ]:
# df.to_csv("s3://amos-training-data/100k_historic_enriched.csv", index=False)

## Create representative dataset

In [ ]:
classes = ["XS", "S", "M", "L", "XL"]
# sample_10k = historic_df.sample(n=10000)
# sample_1k = historic_df.sample(n=10000)


In [ ]:
# excluded_indices = df['Unnamed: 0']
# sample_90k = historic_df.drop(index=excluded_indices)

In [ ]:
# sample_90k.to_csv("s3://amos-training-data/90k_historic_sample.csv")
# sample_10k.to_csv("s3://amos-training-data/10k_historic_sample.csv")
# sample_1k.to_csv("s3://amos-training-data/1k_historic_sample.csv")

## Data exploration (historic)

In [ ]:
historic_df.info()

### Distribution of 'MCC Level'

In [ ]:
mcc_count = historic_df.value_counts('MCC Level', ascending=False)

mcc_count.index = map(lambda x: x[:30], mcc_count.index)

mcc_count.plot(kind='barh', figsize = (30,20), fontsize=20)

### Distribution of 'Merchant size'

In [ ]:
ms_count = historic_df.value_counts('MerchantSizeByDPV', ascending=False)

ms_count.plot(kind='bar')

## Data Exploration (enriched)

In [ ]:
df.info()

### Confidence in Google results

In [ ]:
conf_count = df.value_counts('google_places_confidence', sort=False)
conf_count = pd.DataFrame({'count': conf_count, 'cumulative': conf_count.cumsum()}, index = conf_count.index)
conf_count.plot(kind="bar")
plt.show()

### Distribution of 'business_name_similarity'

In [ ]:
#df = df.dropna()

sim = df.apply(lambda lead: similarity(lead['Company Name'], lead['google_places_name']), axis=1)
sim
df['business_name_similarity'] = sim

In [ ]:
plt.hist(df['business_name_similarity'])
plt.show()

### Correlation between "business name similarity" and confidence in results?

In [ ]:
df_conf = df.groupby(['google_places_confidence']).agg(
    mean=pd.NamedAgg(column="business_name_similarity", aggfunc="mean"),
    median=pd.NamedAgg(column="business_name_similarity", aggfunc="median"),
)
df_conf.plot(kind='line', ylabel='business_name_similarity')
plt.show()

### Comparision of company name and Google business name (confidence >= 0.8)

In [ ]:
df[df['google_places_confidence']>= 0.8][['domain', 'Company Name', 'google_places_name', 'google_places_confidence', 'business_name_similarity']].head(20)

### Comparision of company name and Google business name (confidence < 0.5)

In [ ]:
df[df['google_places_confidence']<0.5][['domain', 'Company Name', 'google_places_name', 'google_places_confidence', 'business_name_similarity']].head(20)

### Correlation between key metrics and business size?

In [ ]:
# select entries with a google candidate and >= 1 reviews
df_google = df.dropna(subset=['google_places_place_id','google_places_rating'])
df_google = df_google[df_google['google_places_user_ratings_total'] != 0]
df_google.info()

In [ ]:
order = ['XS', 'S', 'M', 'L', 'XL']
aggregations = {
    'google_places_user_ratings_total': ['mean', 'median'],
    'google_places_confidence': ['mean', 'median'],
    'regional_atlas_regional_score': ['mean', 'median'],
    'google_places_rating': ['mean', 'median']
}
df_grouped = df_google.groupby(['MerchantSizeByDPV']).agg(aggregations).reset_index()

# Use a trick to order merchant sizes in the table created by groupby
mapping = {size: i for i, size in enumerate(order)}
key = df_grouped['MerchantSizeByDPV'].map(mapping)    
df_grouped = df_grouped.iloc[key.argsort()]

In [ ]:
df_grouped.info()

In [ ]:
fig, axs = plt.subplots(nrows=2,ncols=2, figsize=(8,8), sharex=True)
df_grouped.plot(kind='line', x='MerchantSizeByDPV', y='google_places_user_ratings_total', legend=True,
                ylabel="google_places_user_ratings_total", title="Number of ratings (Google)", ax=axs[0,0], secondary_y=('google_places_user_ratings_total', 'median'))
df_grouped.plot(kind='line', x='MerchantSizeByDPV', y='google_places_rating', legend=False,
                ylabel="google_places_rating", title="Star rating (Google)", ax=axs[0,1], secondary_y=('google_places_rating', 'median'))
df_grouped.plot(kind='line', x='MerchantSizeByDPV', y='google_places_confidence', legend=False,
                ylabel="google_places_confidence", title="Confidence in result (Google)", ax=axs[1,0], secondary_y=('google_places_confidence', 'median'))
df_grouped.plot(kind='line', x='MerchantSizeByDPV', y='regional_atlas_regional_score', legend=False,
                ylabel="regional_atlas_regional_score", title="Regional score (Regionalatlas)", ax=axs[1,1], secondary_y=('regional_atlas_regional_score', 'median'))
fig.tight_layout()


plt.show()

In [ ]:
df_grouped_no_agg = df_google.groupby(['MerchantSizeByDPV'])
df_grouped_no_agg[['google_places_rating']].boxplot(subplots=False, rot=45)

In [ ]:
df_grouped_no_agg[['regional_atlas_regional_score']].boxplot(subplots=False, rot=45)

In [ ]:
df[['google_places_confidence']].boxplot()

## Geolocation analysis

In [ ]:
!pip install plotly
!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim
import time
from tqdm import tqdm
import plotly.express as px

SAMPLE_SIZE = 10

In [ ]:
geo_df = df.sample(SAMPLE_SIZE)

In [ ]:
app = Nominatim(user_agent="test")

In [ ]:
def get_location_by_address(address):
    """This function returns a location as [lat, lon] from an address
    will repeat until success"""
    if address is None or address == "":
        return pd.Series([None, None])
    time.sleep(1)
    try:
        return pd.Series([app.geocode(address).raw['lat'], app.geocode(address).raw['lon']])
    except Exception as e:
        print(e)
        return pd.Series([None, None])

In [ ]:
tqdm.pandas(desc="Geocoding addresses")
geo_df[["lat", "lon"]] = geo_df.progress_apply(lambda lead: get_location_by_address(lead["google_places_formatted_address"]), axis=1)

In [ ]:
geo_df[['lat', 'lon']]

In [ ]:
fig = px.scatter_geo(geo_df,
                    lat=geo_df.lat,
                    lon=geo_df.lon,
                    hover_name="google_places_formatted_address",
                    hover_data=["google_places_name", "Company Name"],
                    color="regional_atlas_regional_score")
fig.show()

## Mapping google business type to MCC Level

In [ ]:
df[['google_places_detailed_type', 'MCC Level']].head(10)

In [ ]:
mcc_df = df.dropna(subset=['google_places_detailed_type'])

In [ ]:
mcc_map = pd.DataFrame(columns=['google_type', 'mcc_level'])

for index, row in mcc_df.iterrows():
    if row['google_places_detailed_type'] is None or row['MCC Level'] is None:
        continue

    google_types = row['google_places_detailed_type'].strip('][').split(', ')
    mapping = []
    for g_type in google_types:
        mapping.append([g_type.strip('\''), row['MCC Level']])
    mcc_map = pd.concat([pd.DataFrame(mapping, columns=mcc_map.columns), mcc_map], ignore_index=True)

In [ ]:
mcc_map.head()

In [ ]:
mcc_map['value'] = 1
mcc_pivot = pd.pivot_table(mcc_map, values='value',index='google_type', columns='mcc_level', aggfunc='count')
mcc_pivot.head(10)

In [ ]:
import numpy as np
fig, ax = plt.subplots(figsize=(20,30))
im = ax.imshow(mcc_pivot)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(mcc_pivot.columns)), labels=mcc_pivot.columns)
ax.set_yticks(np.arange(len(mcc_pivot.index)), labels=mcc_pivot.index)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

fig.tight_layout()
plt.show()

In [ ]:
## export list of google business types
#business_types = pd.DataFrame(mcc_pivot.index)
#business_types['mcc_level'] = None

#business_types.to_csv('s3://amos-training-data/google_business_types.csv', index=False)